In [11]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
data.shape

(20800, 5)

In [5]:
data.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df = data.copy()

In [7]:
df.dropna(inplace=True)

In [8]:
df.reset_index(inplace=True)

In [9]:
df.head(10)

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
8,10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
9,11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


**Text Preprocessing**

In [10]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [12]:
ps = PorterStemmer()
corpus = []

for i in range(0, len(df)):
    text = re.sub('[^a-zA-z]', ' ',df['title'][i])
    text = text.lower()
    text = text.split()
    text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    corpus.append(text)

In [15]:
corpus[12]

'ye paid govern troll social media blog forum websit'

**Creating Applying Bag of Words Model**

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, ngram_range=(1,3))
X = cv.fit_transform(corpus).toarray()

In [17]:
Y = df['label']

In [25]:
cv.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': 5000,
 'min_df': 1,
 'ngram_range': (1, 3),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [27]:
cv.get_feature_names()[:30]

['abandon',
 'abc',
 'abc news',
 'abduct',
 'abe',
 'abedin',
 'abl',
 'abort',
 'abroad',
 'absolut',
 'abstain',
 'absurd',
 'abus',
 'abus new',
 'abus new york',
 'academi',
 'accept',
 'access',
 'access pipelin',
 'access pipelin protest',
 'accid',
 'accident',
 'accord',
 'account',
 'accus',
 'achiev',
 'acknowledg',
 'acknowledg emf',
 'acknowledg emf damag',
 'acquit']

In [22]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state = 42)

**Using Multinomial Naive Bayes**

In [31]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,accuracy_score
classifier=MultinomialNB()

In [33]:
classifier.fit(X_train, Y_train)
pred = classifier.predict(X_test)
score = accuracy_score(Y_test, pred)
print("accuracy:   %0.3f" % score)
cm =confusion_matrix(Y_test, pred)
print(cm)
#plot_confusion_matrix(cm, classes=['FAKE', 'REAL'])

accuracy:   0.894
[[1849  233]
 [ 154 1421]]


**Using Passive Aggressive Classifier Algorithm**

In [36]:
from sklearn.linear_model import PassiveAggressiveClassifier
clf = PassiveAggressiveClassifier(max_iter=50)
clf.fit(X_train,Y_train)

PassiveAggressiveClassifier(max_iter=50)

In [38]:
pred = clf.predict(X_test)
score = accuracy_score(Y_test, pred)
print("accuracy:   %0.3f" % score)
cm =confusion_matrix(Y_test, pred)
print(cm)

accuracy:   0.911
[[1888  194]
 [ 132 1443]]


**Lets Make TF-IDf Model**

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(max_features=5000, ngram_range=(1,3))
X = cv.fit_transform(corpus).toarray()

In [43]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state = 42)

In [44]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,accuracy_score
classifier=MultinomialNB()
classifier.fit(X_train, Y_train)
pred = classifier.predict(X_test)
score = accuracy_score(Y_test, pred)
print("accuracy:   %0.3f" % score)
cm =confusion_matrix(Y_test, pred)
print(cm)

accuracy:   0.882
[[1955  127]
 [ 305 1270]]


In [45]:
from sklearn.linear_model import PassiveAggressiveClassifier
clf = PassiveAggressiveClassifier(max_iter=50)
clf.fit(X_train,Y_train)
pred = clf.predict(X_test)
score = accuracy_score(Y_test, pred)
print("accuracy:   %0.3f" % score)
cm =confusion_matrix(Y_test, pred)
print(cm)

accuracy:   0.911
[[1881  201]
 [ 124 1451]]
